In [1]:
import ast
import json
import string
import re
import joblib
from pathlib import Path

import pandas as pd
import numpy as np

# from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from mittens import GloVe

#from langchain.vectorstores import Pinecone
import pinecone

In [2]:
pd.set_option('display.max_columns', None)

## 1. Preparation

### <span style="color:green">Load df</span>

In [3]:
# Path object pointing to the project's root
df_path = Path(Path.cwd().parent)

# Path for the CSV file
df_path = df_path.joinpath('data', 'beops_papers.csv')

results_df = pd.read_csv(df_path, index_col=0)
results_df.head(2)

,@_fa,link,prism:url,dc:identifier,eid,dc:title,dc:creator,prism:publicationName,prism:issn,prism:volume,prism:pageRange,prism:coverDate,prism:coverDisplayDate,prism:doi,pii,dc:description,citedby-count,affiliation,prism:aggregationType,subtype,subtypeDescription,author-count,author,authkeywords,article-number,source-id,fund-acr,fund-no,fund-sponsor,openaccess,openaccessFlag,prism:eIssn,prism:issueIdentifier,freetoread,freetoreadLabel
0,True,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85141444854,2-s2.0-85141444854,Pricing decisions for a social comparison prod...,Zhang M.,Transportation Research Part E: Logistics and ...,13665545,168,NaN,2022-12-01,December 2022,10.1016/j.tre.2022.102934,S1366554522003118,As a natural psychological tendency of human b...,0,"[{'@_fa': 'true', 'affiliation-url': 'https://...",Journal,ar,Article,"{'@limit': '100', '@total': '4', '$': '4'}","[{'@_fa': 'true', '@seq': '1', 'author-url': '...",Behavioral operations management | Pricing dec...,102934.0,20909,NSFC,72071188,National Natural Science Foundation of China,0,False,NaN,NaN,NaN,NaN
1,True,"[{'@_fa': 'true', '@ref': 'self', '@href': 'ht...",https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85128739402,2-s2.0-85128739402,Quantifying heterogeneity in human behavior: A...,Loske D.,Logistics Research,1865035X,15,1-17,2022-12-01,December 2022,10.23773/2022_1,NaN,Human operators will remain to play an essenti...,0,"[{'@_fa': 'true', 'affiliation-url': 'https://...",Journal,ar,Article,"{'@limit': '100', '@total': '3', '$': '3'}","[{'@_fa': 'true', '@seq': '1', 'author-url': '...",behavioral operations management | forklift op...,1.0,19400158510,NaN,undefined,NaN,0,False,18650368,1,NaN,NaN


### Title

In [4]:
# Some titles have non-breaking spaces (NBSP). We replace them by a normal space, i.e., ' '
    # https://cookierobotics.com/045/
# A title has 'R&amp;D', which corresponds to 'R&D'. We make the corresponding replacement
    
series = results_df['dc:title'].str.replace('\xa0', " ").str.replace('R&amp;D', 'R&D').str.strip()
series.head()

0    Pricing decisions for a social comparison prod...
1    Quantifying heterogeneity in human behavior: A...
2    Retailer Inventory Sharing in Two-Tier Supply ...
3              Fairness ideals in inventory allocation
4    The Impacts of Algorithmic Work Assignment on ...
Name: dc:title, dtype: object

In [5]:
# series.to_csv('test.csv')

### Keywords

In [6]:
series = results_df['authkeywords'].str.replace('R&amp;D', 'R&D')
series.head()

0    Behavioral operations management | Pricing dec...
1    behavioral operations management | forklift op...
2    behavioral operations | inventory sharing | ri...
3    behavioral operations | experimental economics...
4    artificial intelligence | behavioral operation...
Name: authkeywords, dtype: object

In [7]:
# series.to_csv('test.csv')

### Authors

In [8]:
# Function to extract 'surname' and 'given-name' from the list of dictionaries
def extract_names(data: pd.Series) -> str:
    """
    This function extracts the authors names. They're stored in a list of dictionaries,
    where each dictionary stores the info from an author. Note that the list is stored
    as a string.

    Args:
        data: pd.Series with strings, each containing a list of dictionaries.
    
    Returns:
        author_list: string with the authors names. The string contains each author's
            surname and given name separated by ',', while each author is separated by
            ';'. 
    """
    
    # Extract the list
    data_list = ast.literal_eval(data)
    
    # Create the string that will store the authors names
    author_list = ''
    
    # Loop through each item (i.e., dictionary) in the list
    for item in data_list:
        author_list += item.get('surname') + ', ' + item.get('given-name') + '; '
    
    # Remove the trailing '; '
    author_list = author_list[:-2] 
    
    return author_list

In [9]:
series = results_df['author'].apply(extract_names)
series.head()

0    Zhang, Mengyao; Gou, Qinglong; Yu, Lili; Zhang...
1              Loske, D.; Klumpp, M.; Klumpp, Matthias
2    Davis, Andrew M.; Huang, Rihuan; Thomas, Dougl...
3                  Spiliotopoulou, Eirini; Conte, Anna
4    Bai, Bing; Dai, Hengchen; Zhang, Dennis J.; Zh...
Name: author, dtype: object

In [10]:
# series.to_csv('test.csv')

### Journal

In [11]:
series = results_df['prism:publicationName'].str.replace(r'\(.*\)', '', regex=True).str.strip()
series.head()

0    Transportation Research Part E: Logistics and ...
1                                   Logistics Research
2                                   Management Science
3                                    Decision Sciences
4      Manufacturing and Service Operations Management
Name: prism:publicationName, dtype: object

In [12]:
# series.to_csv('test.csv')

### Publication year

In [13]:
series = results_df['prism:coverDate'].str[:4].astype(int)
series.head()

0    2022
1    2022
2    2022
3    2022
4    2022
Name: prism:coverDate, dtype: int64

In [14]:
# series.to_csv('test.csv')

### doi

In [15]:
series = 'https://doi.org/' + results_df['prism:doi']
series.head()

0    https://doi.org/10.1016/j.tre.2022.102934
1              https://doi.org/10.23773/2022_1
2       https://doi.org/10.1287/mnsc.2022.4323
3           https://doi.org/10.1111/deci.12540
4       https://doi.org/10.1287/msom.2022.1120
Name: prism:doi, dtype: object

In [16]:
# series.to_csv('test.csv')

### Abstract

In [17]:
# We replace non-breaking spaces (NBSP) by a normal space, i.e., ' '
    # https://cookierobotics.com/045/
# Some abstracts have copyright information. We remnove this info

series = results_df['dc:description'].str.replace('\xa0', " ").str.replace(r' (©|Copyright).*', '', regex=True).str.strip()
series.head()

0    As a natural psychological tendency of human b...
1    Human operators will remain to play an essenti...
2    When multiple retailers hold inventory to sati...
3    We study fairness ideals in distribution syste...
4    Problem definition: We study how algorithmic (...
Name: dc:description, dtype: object

In [18]:
# series.to_csv('test.csv')

### <span style="color:green">Bring all together</span>

In [4]:
def extract_names(data: pd.Series) -> str:
    """
    This function extracts the author names (given-name and surname). They're stored in
    a list of dictionaries, where each dictionary stores the info from an author. Note
    that the list is stored as a string.

    Args:
        data: pd.Series with strings, each containing a list of dictionaries.
    
    Returns:
        author_list: string with the author names. The string contains each author's
            surname and given name separated by ',', while each author is separated by
            ';'. 
    """
    
    # Extract the list
    data_list = ast.literal_eval(data)
    
    # Create the string that will store the author names
    author_list = ''
    
    # Loop through each item (i.e., dictionary) in the list
    for item in data_list:
        author_list += item.get('surname') + ', ' + item.get('given-name') + '; '
    
    # Remove the trailing '; '
    author_list = author_list[:-2] 
    
    return author_list

In [5]:
def clean_df(results_df: pd.DataFrame) -> pd.DataFrame:
    """
    This function cleans the title, keywords, author names, journal, publication date,
    doi and abstract retrieved from the Scopus search.

    Args:
        results_df: pd.DataFrame with the Scopus search results.

    Returns:
        clean_results_df: pd.DataFrame with the clean title, keywords, author names,
            journal, publication date, doi and abstract.     
    """

    # Title
        # Some titles have non-breaking spaces (NBSP). We replace them by a normal space, i.e., ' '
        # A title has 'R&amp;D', which corresponds to 'R&D'. We make the corresponding replacement
    title_series = results_df['dc:title'].str.replace(
        '\xa0',
        " "
        ).str.replace(
            'R&amp;D',
            'R&D'
            ).str.strip()
    title_series.rename('title', inplace=True)

    # Keywords
        # A keyword has 'R&amp;D', which corresponds to 'R&D'. We make the corresponding replacement
    keyword_series = results_df['authkeywords'].str.replace('R&amp;D', 'R&D')
    keyword_series.rename('keywords', inplace=True)

    # Authors
    authors_series = results_df['author'].apply(extract_names)
    authors_series.rename('authors', inplace=True)
    
    # Journal
        # Some journals have a country name inside parentheses. We remove such info from the name
    journal_series = results_df['prism:publicationName'].str.replace(
        r'\(.*\)',
        '',
        regex=True
        ).str.strip()
    journal_series.rename('journal', inplace=True)

    # Publication year
    pubyear_series = results_df['prism:coverDate'].str[:4].astype(int)
    pubyear_series.rename('publication_year', inplace=True)

    # doi
    doi_series = 'https://doi.org/' + results_df['prism:doi']
    doi_series.rename('doi', inplace=True)

    # Abstract
        # Some abstracts have non-breaking spaces (NBSP). We replace them by a normal space, i.e., ' '
        # Some abstracts have copyright information. We remnove this info
    abstract_series = results_df['dc:description'].str.replace(
        '\xa0',
        " "
        ).str.replace(
            r' (©|Copyright).*',
            '',
            regex=True
            ).str.strip()
    abstract_series.rename('abstract', inplace=True)
    
    clean_results_df = pd.concat(
        [title_series,
         keyword_series,
         authors_series,
         journal_series,
         pubyear_series,
         doi_series,
         abstract_series],
         axis=1
    )

    return clean_results_df

In [6]:
papers_df = clean_df(results_df)
papers_df.head()

,title,keywords,authors,journal,publication_year,doi,abstract
0,Pricing decisions for a social comparison prod...,Behavioral operations management | Pricing dec...,"Zhang, Mengyao; Gou, Qinglong; Yu, Lili; Zhang...",Transportation Research Part E: Logistics and ...,2022,https://doi.org/10.1016/j.tre.2022.102934,As a natural psychological tendency of human b...
1,Quantifying heterogeneity in human behavior: A...,behavioral operations management | forklift op...,"Loske, D.; Klumpp, M.; Klumpp, Matthias",Logistics Research,2022,https://doi.org/10.23773/2022_1,Human operators will remain to play an essenti...
2,Retailer Inventory Sharing in Two-Tier Supply ...,behavioral operations | inventory sharing | ri...,"Davis, Andrew M.; Huang, Rihuan; Thomas, Dougl...",Management Science,2022,https://doi.org/10.1287/mnsc.2022.4323,When multiple retailers hold inventory to sati...
3,Fairness ideals in inventory allocation,behavioral operations | experimental economics...,"Spiliotopoulou, Eirini; Conte, Anna",Decision Sciences,2022,https://doi.org/10.1111/deci.12540,We study fairness ideals in distribution syste...
4,The Impacts of Algorithmic Work Assignment on ...,artificial intelligence | behavioral operation...,"Bai, Bing; Dai, Hengchen; Zhang, Dennis J.; Zh...",Manufacturing and Service Operations Management,2022,https://doi.org/10.1287/msom.2022.1120,Problem definition: We study how algorithmic (...


In [7]:
def nlp_preparation(abstract: str) -> str:
    """
    This function prepares an abstract or an user's input for the embedding. It removes
    punctuation from the abstract, lower cases it, removes stopwords from it and
    lemmatizes it.

    Args:
        abstract: string with the abstract content.

    Returns:
        prepared_abstract: string prepared for the embedding. 
    """

    # Replace hyphens, en-dashes and em-dashes with a space
    prepared_abstract = re.sub(r'[-\u2013\u2014]', ' ', abstract)

    # Remove punctuation
    prepared_abstract = prepared_abstract.translate(
        str.maketrans(
            '',
            '',
            string.punctuation
        )
    )

    # Lower case
    prepared_abstract = prepared_abstract.lower()

    # Get list of English stopwords
    ENG_STOPWORDS = stopwords.words('english')
    # Remove stopwords
    prepared_abstract = ' '.join(
        [word for word in prepared_abstract.split() if word not in ENG_STOPWORDS]
    )

    # Lemmatizer object
    lemmatizer = WordNetLemmatizer()
    # Lemmatization on all rows
    prepared_abstract = ' '.join(
        lemmatizer.lemmatize(word) for word in prepared_abstract.split()
    )
    
    return prepared_abstract

In [8]:
papers_df['abstract_lem'] = papers_df['abstract'].apply(lambda x: nlp_preparation(x))
papers_df.head()

,title,keywords,authors,journal,publication_year,doi,abstract,abstract_lem
0,Pricing decisions for a social comparison prod...,Behavioral operations management | Pricing dec...,"Zhang, Mengyao; Gou, Qinglong; Yu, Lili; Zhang...",Transportation Research Part E: Logistics and ...,2022,https://doi.org/10.1016/j.tre.2022.102934,As a natural psychological tendency of human b...,natural psychological tendency human being soc...
1,Quantifying heterogeneity in human behavior: A...,behavioral operations management | forklift op...,"Loske, D.; Klumpp, M.; Klumpp, Matthias",Logistics Research,2022,https://doi.org/10.23773/2022_1,Human operators will remain to play an essenti...,human operator remain play essential role pick...
2,Retailer Inventory Sharing in Two-Tier Supply ...,behavioral operations | inventory sharing | ri...,"Davis, Andrew M.; Huang, Rihuan; Thomas, Dougl...",Management Science,2022,https://doi.org/10.1287/mnsc.2022.4323,When multiple retailers hold inventory to sati...,multiple retailer hold inventory satisfy rando...
3,Fairness ideals in inventory allocation,behavioral operations | experimental economics...,"Spiliotopoulou, Eirini; Conte, Anna",Decision Sciences,2022,https://doi.org/10.1111/deci.12540,We study fairness ideals in distribution syste...,study fairness ideal distribution system inven...
4,The Impacts of Algorithmic Work Assignment on ...,artificial intelligence | behavioral operation...,"Bai, Bing; Dai, Hengchen; Zhang, Dennis J.; Zh...",Manufacturing and Service Operations Management,2022,https://doi.org/10.1287/msom.2022.1120,Problem definition: We study how algorithmic (...,problem definition study algorithmic versus hu...


In [9]:
papers_df.loc[0,'abstract_lem']

'natural psychological tendency human being social comparison pervasive among consumer however plenty literature studied role social comparison affecting consumer psychology empirically focus problem social comparison supply chain paper proposes modified utility model consumer intrinsic social value social comparison product furthermore social value incorporates two different consumer psychology social comparison assimilate effect upward comparison contrast effect downward comparison based model derive firm price decision social comparison product supply chain consisting manufacturer retailer two case centralized case decentralized case find several interesting pattern consumers’ behavior presence social comparison also find consumer highly sensitive social comparison supply chain coordinated spontaneously using simple wholesale price contract moreover even consumer sensitive social comparison wholesale retail price decentralized supply chain significantly reduce compared non social co

If you want to perform stemming instead of lemmatization, first, import the Porter Stemmer:

```python
from nltk.stem.porter import PorterStemmer
```

Then, replace the lemmatization code with the following code:

```python
# Stemmer object
stemmer = PorterStemmer()
# Stemming all rows
abstract_r = ' '.join(
    stemmer.stem(word) for word in abstract_r.split()
)
```

Make sure to also update the name of the prepared abstract column:

```python
papers_df['abstract_stem'] = papers_df['abstract'].apply(lambda x: nlp_preparation(x))
```

Finally, the code below tokenizes the prepared abstract:

```python
tokens_lem = papers_df['abstract_lem'].apply(word_tokenize)
# tokens_lem = papers_df['abstract_stem'].apply(word_tokenize)
```

## <span style="color:green">2. Embeddings using TF-IDF</span>

### <span style="color:green">Abtracts</span>

In [10]:
# Instantiate TfidfVectorizer object
tf_idf_vec = TfidfVectorizer(max_features=128, ngram_range=(1,2))

# Get abstracts prepared with lemmatization
corpus_lem = papers_df['abstract_lem']

# Retrieve the first two lemmatized abstracts
# corpus_lem[:2]

In [11]:
# Generating TF-IDF
tf_idf = tf_idf_vec.fit_transform(corpus_lem)

# Getting the reviews decoded by the TF-IDF
# This isn't necessary, it's just to check how this looks like
example_tf_idf = pd.DataFrame(
    tf_idf.toarray(),
    columns=tf_idf_vec.get_feature_names_out()
)

example_tf_idf.shape

(282, 128)

In [12]:
example_tf_idf.head()

,affect,also,analysis,aversion,based,behavior,behavioral,behaviour,benefit,bias,buyer,case,chain,concern,condition,consumer,context,contract,cost,customer,data,decision,decision maker,decision making,demand,design,different,effect,efficiency,effort,expected,experiment,experimental,factor,fairness,field,find,finding,firm,first,group,high,higher,however,human,impact,implication,important,improve,incentive,increase,individual,influence,information,inventory,investigate,laboratory,lead,level,literature,low,make,maker,making,management,manager,managerial,may,mechanism,method,model,new,newsvendor,one,operation,operation management,operational,optimal,order,order quantity,ordering,paper,patient,performance,policy,practice,preference,price,problem,process,product,profit,project,quality,quantity,rate,relationship,research,response,result,retailer,risk,role,service,setting,sharing,show,significantly,social,strategy,study,subject,suggest,supplier,supply,supply chain,system,task,theory,time,two,type,use,used,using,value,well,worker
0,0.000000,0.065054,0.000000,0.0,0.038359,0.031311,0.031311,0.0,0.090001,0.0,0.0,0.142790,0.170940,0.000000,0.048422,0.390741,0.000000,0.047199,0.000000,0.0,0.000000,0.026426,0.000000,0.000000,0.000000,0.00000,0.042470,0.095723,0.000000,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.063815,0.000000,0.045347,0.043373,0.000000,0.0,0.000000,0.039054,0.041615,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.080567,0.0,0.0,0.000000,0.000000,0.039781,0.000000,0.045000,0.000000,0.048004,0.000000,0.062331,0.048852,0.0,0.000000,0.038819,0.048004,0.00000,0.0,0.000000,0.0,0.0,0.072476,0.0,0.000000,0.0,0.0,0.0,0.141597,0.038359,0.000000,0.132989,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.092865,0.0,0.045000,0.0,0.000000,0.000000,0.000000,0.045347,0.7755,0.000000,0.000000,0.000000,0.000000,0.0,0.163427,0.173608,0.000000,0.00000,0.000000,0.000000,0.066342,0.0,0.000000,0.000000,0.035088,0.104456,0.000000,0.000000
1,0.000000,0.000000,0.089996,0.0,0.082955,0.203140,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.073934,0.000000,0.000000,0.000000,0.000000,0.000000,0.084458,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.069003,0.080586,0.000000,0.093798,0.000000,0.0,0.000000,0.084458,0.359984,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.089401,0.0,0.0,0.000000,0.000000,0.0,0.0,0.409940,0.087117,0.0,0.0,0.000000,0.000000,0.172058,0.000000,0.000000,0.000000,0.000000,0.115349,0.000000,0.000000,0.0,0.000000,0.335800,0.000000,0.10859,0.0,0.217358,0.0,0.0,0.000000,0.0,0.294232,0.0,0.0,0.0,0.000000,0.000000,0.196132,0.000000,0.000000,0.0,0.0,0.093798,0.000000,0.0,0.000000,0.0,0.056915,0.000000,0.0,0.097317,0.0,0.000000,0.000000,0.068031,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.070685,0.075089,0.289753,0.10859,0.084974,0.281393,0.071735,0.0,0.090601,0.099614,0.000000,0.000000,0.093798,0.000000
2,0.065046,0.091133,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.143680,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.074039,0.000000,0.000000,0.255226,0.00000,0.059495,0.000000,0.000000,0.070343,0.0,0.000000,0.00000,0.0,0.078198,0.000000,0.089398,0.000000,0.063525,0.000000,0.000000,0.0,0.059089,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.055045,0.000000,0.0,0.0,0.347473,0.059089,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.111456,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.050765,0.0,0.047650,0.0,0.0,0.0,0.198361,0.000000,0.000000,0.000000,0.160264,0.0,0.0,0.060760,0.000000,0.0,0.000000,0.0,0.110606,0.650464,0.0,0.000000,0.0,0.053110,0.229177,0.000000,0.000000,0.0000,0.278755,0.000000,0.000000,0.000000,0.0,0.183154,0.145923,0.000000,0.00000,0.055045,0.000000,0.139406,0.0,0.058690,0.000000,0.000000,0.000000,0.060760,0.000000
3,0.069738,0.00

In [13]:
tf_idf

<282x128 sparse matrix of type '<class 'numpy.float64'>'
	with 7101 stored elements in Compressed Sparse Row format>

In [14]:
tf_idf[0,:]

<1x128 sparse matrix of type '<class 'numpy.float64'>'
	with 39 stored elements in Compressed Sparse Row format>

In [15]:
row_values = example_tf_idf.loc[0]  # Extract the specific row
sum(row_values > 0)  # Count values greater than 0

39

We save the vectorizer.

In [16]:
model_path = Path(Path.cwd().parent)
print(model_path)

/mnt/c/Users/USER/DS_Projects/nlp_paper_recommender


In [17]:
model_path = model_path.joinpath('models/tfidf_model.joblib')
joblib.dump(tf_idf_vec, model_path)

['/mnt/c/Users/USER/DS_Projects/nlp_paper_recommender/models/tfidf_model.joblib']

### User input

In [18]:
user_input = "supply chain contract design considering decision maker behavior"

In [19]:
user_input_prepared = nlp_preparation(user_input)
user_input_prepared

'supply chain contract design considering decision maker behavior'

We load the vectorizer.

In [20]:
loaded_tfidf_vec = joblib.load(model_path)

We vectorize the input.

In [21]:
user_input_tf_idf = loaded_tfidf_vec.transform([user_input_prepared])
user_input_tf_idf.shape

(1, 128)

### Similarity score

In [22]:
# Calculate cosine similarity between user input and abstracts
similarities = cosine_similarity(user_input_tf_idf, example_tf_idf)

In [23]:
# Create a list of (paper_index, similarity_score) tuples
similarity_scores = list(enumerate(similarities[0]))

In [24]:
# Sort the list by similarity score in descending order
similarity_scores.sort(key=lambda x: x[1], reverse=True)

In [25]:
# Get the top N papers (e.g., top 10)
TOP_N = 10
top_papers_tf_idf = similarity_scores[:TOP_N]
top_papers_tf_idf

[(211, 0.5657141381232188),
 (224, 0.5615318670172559),
 (247, 0.5428019210807384),
 (246, 0.490500032036113),
 (183, 0.46999268018039),
 (241, 0.46791249380506794),
 (73, 0.46169240302807246),
 (235, 0.45934964982354143),
 (44, 0.4517904067819405),
 (12, 0.44138576774310123)]

In [26]:
# Extract the indices from the top_papers list
top_paper_indices_tf_idf = [index for index, _ in top_papers_tf_idf]
top_paper_indices_tf_idf

[211, 224, 247, 246, 183, 241, 73, 235, 44, 12]

In [27]:
papers_df.iloc[top_paper_indices_tf_idf]

,title,keywords,authors,journal,publication_year,doi,abstract,abstract_lem
211,Decision Making and Cognition in Multi-Echelon...,behavioral operations | bullwhip effect | cogn...,"Narayanan, Arunachalam; Moritz, Brent B.",Production and Operations Management,2015,https://doi.org/10.1111/poms.12343,Supply chain performance often depends on the ...,supply chain performance often depends individ...
224,Supply chain contract design: Impact of bounde...,behavioral operations management | bounded rat...,"Wu, Diana Yan; Chen, Kay Yut",Production and Operations Management,2014,https://doi.org/10.1111/poms.12057,"In this article, we model various forms of non...",article model various form non optimizing beha...
247,The impact of repeated interactions on supply ...,Behavioral operations management | Repeated in...,"Wu, Diana Yan",International Journal of Production Economics,2013,https://doi.org/10.1016/j.ijpe.2012.05.004,"In this laboratory study, we investigate the i...",laboratory study investigate interactive behav...
246,How can we improve the performance of supply c...,Behavioral operations management | Buyback | R...,"Elahi, Ehsan; Lamba, Narasimha; Ramaswamy, Chi...",International Journal of Production Economics,2013,https://doi.org/10.1016/j.ijpe.2012.10.023,Although optimal forms of supply chain contrac...,although optimal form supply chain contract wi...
183,Strategic risk in supply chain contract design,Asymmetric information | Behavioral operations...,"Sadrieh, Abdolkarim; Voigt, Guido",Journal of Business Economics,2017,https://doi.org/10.1007/s11573-015-0790-4,Supply chains facing asymmetric information ca...,supply chain facing asymmetric information eit...
241,Designing buyback contracts for irrational but...,Behavioral operations | Contract optimization ...,"Becker-Peth, Michael; Katok, Elena; Thonemann,...",Management Science,2013,https://doi.org/10.1287/mnsc.1120.1662,One of the main assumptions in research on des...,one main assumption research designing supply ...
73,Network trust and trust behaviors among execut...,Behavior | Behavioral operations | Belief | Ex...,"Choi, Emily W.; Özer, Özalp; Zheng, Yanchong",Management Science,2020,https://doi.org/10.1287/mnsc.2019.3499,We integrate the results of a social network s...,integrate result social network survey forecas...
235,Agent-system co-development in supply chain re...,Adaptive complex systems | Agent-system co-dev...,"Tangpong, Chanchai; Hung, Kuo Ting; Li, Jin",Journal of Operations Management,2014,https://doi.org/10.1016/j.jom.2014.03.002,"In this study, we develop an agent-system co-d...",study develop agent system co development asc ...
44,Corporate social and environmental irresponsib...,Behavioural operations | Intangible resources ...,"Fracarolli Nunes, Mauro; Lee Park, Camila; Shi...",International Journal of Production Economics,2021,https://doi.org/10.1016/j.ijpe.2021.108275,Corporate social irresponsibility (CSI) and co...,corporate social irresponsibility csi corporat...
12,Light in Dark Places: The Hidden World of Supp...,Behavioral operations | ethics | fraud triangl...,"Duhadway, Scott; Talluri, Sri; Ho, William; Bu...",IEEE Transactions on Engineering Management,2022,https://doi.org/10.1109/TEM.2019.2957439,"Interorganizational fraud, or fraud that occur...",interorganizational fraud fraud occurs organiz...


In [28]:
papers_df.loc[211, 'title']

'Decision Making and Cognition in Multi-Echelon Supply Chains: An Experimental Study'

## <span style="color:green">3. Embeddings using GloVe</span>

### Abstracts

Generate a Bag of Words (BoW) using `CountVectorizer` from `scikit-learn`:

In [43]:
# Instantiate CountVectorizer object
count_vectorizer = CountVectorizer(max_features=128, ngram_range=(1,2))

# Generate Bag of Words
bow = count_vectorizer.fit_transform(corpus_lem)

# Get vocabulary
vocab = count_vectorizer.get_feature_names_out()

# Getting the reviews decoded by the BoW
# This isn't necessary, it's just to check how this looks like
example_count_vect = pd.DataFrame(
    bow.toarray(),
    columns=count_vectorizer.get_feature_names_out()
)

example_count_vect.shape

(282, 128)

In [44]:
example_count_vect.head()

,affect,also,analysis,aversion,based,behavior,behavioral,behaviour,benefit,bias,buyer,case,chain,concern,condition,consumer,context,contract,cost,customer,data,decision,decision maker,decision making,demand,design,different,effect,efficiency,effort,expected,experiment,experimental,factor,fairness,field,find,finding,firm,first,group,high,higher,however,human,impact,implication,important,improve,incentive,increase,individual,influence,information,inventory,investigate,laboratory,lead,level,literature,low,make,maker,making,management,manager,managerial,may,mechanism,method,model,new,newsvendor,one,operation,operation management,operational,optimal,order,order quantity,ordering,paper,patient,performance,policy,practice,preference,price,problem,process,product,profit,project,quality,quantity,rate,relationship,research,response,result,retailer,risk,role,service,setting,sharing,show,significantly,social,strategy,study,subject,suggest,supplier,supply,supply chain,system,task,theory,time,two,type,use,used,using,value,well,worker
0,0,2,0,0,1,1,1,0,2,0,0,3,5,0,1,7,0,1,0,0,0,1,0,0,0,0,1,3,0,0,0,0,0,0,0,0,2,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0,1,0,1,0,2,1,0,0,1,1,0,0,0,0,0,2,0,0,0,0,0,3,1,0,3,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,1,15,0,0,0,0,0,5,5,0,0,0,0,2,0,0,0,1,2,0,0
1,0,0,1,0,1,3,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,1,4,0,0,0,0,0,0,1,0,0,0,0,0,0,5,1,0,0,0,0,2,0,0,0,0,1,0,0,0,0,4,0,1,0,3,0,0,0,0,4,0,0,0,0,0,2,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1,3,1,1,3,1,0,1,1,0,0,1,0
2,1,2,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,2,0,0,5,0,1,0,0,1,0,0,0,0,1,0,2,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,1,0,1,0,0,0,3,0,0,0,3,0,0,1,0,0,0,0,3,10,0,0,0,1,3,0,0,0,4,0,0,0,0,4,3,0,0,1,0,3,0,1,0,0,0,1,0
3,1,0,0,0,1,0,0,0,0,0,0,3,4,0,0,0,1,0,0,0,1,2,1,0,6,0,0,0,0,0,0,1,1,0,5,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,6,0,0,0,1,0,0,0,1,0,0,0,0,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,0,0,0,6,0,1,0,2,0,0,0,0,0,1,1,2,0,7,4,1,0,0,0,0,0,0,0,0,0,0,0
4,0,2,0,0,5,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,3,0,3,0,2,0,0,2,0,0,4,0,0,3,2,1,1,0,1,2,0,0,0,10,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,3,0,1,1,2,0,0,0,0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,1,3,1,0,0,0,0,0,0,1,0,3,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,5,0,0,0,0,0,1,0,0,0,6


In [45]:
bow

<282x128 sparse matrix of type '<class 'numpy.int64'>'
	with 7101 stored elements in Compressed Sparse Row format>

Generate the weighted co-occurrence matrix:

In [46]:
# Compute co-occurrence matrix
coocc = (bow.T @ bow).toarray()
np.fill_diagonal(coocc, 0)

# Normalize co-occurrence matrix 
row_sums = coocc.sum(axis=1, keepdims=True)
cooccurrence = coocc / row_sums

Vectorized operation suggested by ChatGPT:

```python
# Compute co-occurrence matrix
coocc = (bow.T @ bow - np.diag(np.diag(bow.T @ bow))).toarray()

# Normalize co-occurrence matrix 
row_sums = coocc.sum(axis=1, keepdims=True)
cooccurrence = coocc / row_sums
```

It raises an error:

```python
ValueError: Input must be 1- or 2-d.
```

ChatGPT suggests a modification when creating the BoW to solve this error:

```python
bow = count_vectorizer.fit_transform(corpus_lem).toarray()
```

It doesn't work either and when told about this error, it suggests again the initial code.

I initially generated the matrix with the following code:

```python
# Compute co-occurrence matrix
N = len(vocab)
coocc = np.zeros((N, N))

for i in range(N):
    for j in range(N):
        if i != j:
            coocc[i,j] = np.sum(bow[:,i].toarray() * bow[:,j].toarray())

# Normalize co-occurrence matrix
row_sums = coocc.sum(axis=1, keepdims=True)
cooccurrence = coocc / row_sums
```

The vectorized operation works as follows:

- `bow.T @ bow` is the matrix multiplication of the transpose of bow with itself. This is equivalent to the nested loops, but in a more efficient matrix multiplication form.
- `np.diag(np.diag(...))` is used to set the diagonal elements to zero because you don't want to include the self-co-occurrence (i == j) in the matrix.

GloVe:

- [What does dimension represent in GloVe pre-trained word vectors?](https://datascience.stackexchange.com/questions/61692/what-does-dimension-represent-in-glove-pre-trained-word-vectors)

In [47]:
glove_model = GloVe(n=25, max_iter=1000)  # 25 is the embedding dimension
embeddings = glove_model.fit(cooccurrence)

Iteration 10: error 118.5813

Iteration 1000: error 1.2873

In [48]:
embeddings.shape

(128, 25)

### User input

Vectorize the user input text using `CountVectorizer` with the same vocabulary as before:

In [49]:
user_input_bow = count_vectorizer.transform([user_input_prepared])

Use the GloVe embeddings to convert the BoW vectors to dense embedding vectors:

In [50]:
# Embeddings matrix
vocab_list = vocab.tolist()
embedding_matrix = np.vstack([embeddings[vocab_list.index(w)] for w in vocab_list])

In [51]:
# Convert user BoW to embedding  
user_embedding = user_input_bow.dot(embedding_matrix)

Similarly, convert all the paper abstract embeddings:

In [52]:
paper_embeddings = bow.dot(embedding_matrix)

### Similarity score

Compute cosine similarity between user embedding and all paper embeddings:

In [53]:
# Calculate cosine similarity between user input and abstracts
similarities = cosine_similarity(user_embedding, paper_embeddings)

In [54]:
# Create a list of (paper_index, similarity_score) tuples
similarity_scores = list(enumerate(similarities[0]))

In [55]:
# Sort the list by similarity score in descending order
similarity_scores.sort(key=lambda x: x[1], reverse=True)

In [56]:
# Get the top N papers (e.g., top 10)
TOP_N = 10
top_papers_glove = similarity_scores[:TOP_N]
top_papers_glove

[(25, 0.9106458322171878),
 (73, 0.8996654499636186),
 (22, 0.8905082589509502),
 (211, 0.876241082404299),
 (247, 0.8664214155713326),
 (223, 0.8615395669748862),
 (242, 0.8603340551374768),
 (115, 0.8392316640413046),
 (104, 0.8387429465784567),
 (3, 0.8372247528261774)]

In [57]:
# Extract the indices from the top_papers list
top_paper_indices_glove = [index for index, _ in top_papers_glove]
top_paper_indices_glove

[25, 73, 22, 211, 247, 223, 242, 115, 104, 3]

In [58]:
papers_df.iloc[top_paper_indices_glove]

,title,keywords,authors,journal,publication_year,doi,abstract,abstract_lem
25,Mitigating behavioral supply risk under dual s...,behavioral operations management | dual sourci...,"Xue, Chao; Wu, Yan; Zhu, Wanshan; Zhao, Xiaobo...",Production and Operations Management,2022,https://doi.org/10.1111/poms.13644,We consider an order allocation game in which ...,consider order allocation game manufacturer so...
73,Network trust and trust behaviors among execut...,Behavior | Behavioral operations | Belief | Ex...,"Choi, Emily W.; Özer, Özalp; Zheng, Yanchong",Management Science,2020,https://doi.org/10.1287/mnsc.2019.3499,We integrate the results of a social network s...,integrate result social network survey forecas...
22,Unraveling Behavioral Ordering: Relative Costs...,behavioral operations | bullwhip effect | inve...,"Moritz, Brent B.; Narayanan, Arunachalam; Park...",Manufacturing and Service Operations Management,2022,https://doi.org/10.1287/msom.2021.1030,Problem definition: We study the bullwhip effe...,problem definition study bullwhip effect analy...
211,Decision Making and Cognition in Multi-Echelon...,behavioral operations | bullwhip effect | cogn...,"Narayanan, Arunachalam; Moritz, Brent B.",Production and Operations Management,2015,https://doi.org/10.1111/poms.12343,Supply chain performance often depends on the ...,supply chain performance often depends individ...
247,The impact of repeated interactions on supply ...,Behavioral operations management | Repeated in...,"Wu, Diana Yan",International Journal of Production Economics,2013,https://doi.org/10.1016/j.ijpe.2012.05.004,"In this laboratory study, we investigate the i...",laboratory study investigate interactive behav...
223,Order stability in supply chains: Coordination...,beer distribution game | behavioral operations...,"Croson, Rachel; Donohue, Karen; Katok, Elena; ...",Production and Operations Management,2014,https://doi.org/10.1111/j.1937-5956.2012.01422.x,The bullwhip effect describes the tendency for...,bullwhip effect describes tendency variance or...
242,"Durability, transit lags, and optimality of in...",behavioral operations | decision biases | inve...,"Bloomfield, Robert J.; Kulp, Susan L.",Production and Operations Management,2013,https://doi.org/10.1111/poms.12017,Two laboratory experiments on a single-echelon...,two laboratory experiment single echelon inven...
115,A behavioral investigation of supply chain con...,Behavioral operations | Development operations...,"Castañeda, Jaime Andrés; Brennan, Mark; Goentz...",International Journal of Production Economics,2019,https://doi.org/10.1016/j.ijpe.2018.12.024,The business context in developing economies i...,business context developing economy introduces...
104,Analysis of Bullwhip effect: A Behavioral Appr...,beer distribution game | behavioural operation...,"Khan, Maaz Hasan; Ahmed, Salma; Hussain, Danish",Supply Chain Forum,2019,https://doi.org/10.1080/16258312.2019.1661756,Bullwhip effect (BWE) in a supply chain is a p...,bullwhip effect bwe supply chain phenomenon wh...
3,Fairness ideals in inventory allocation,behavioral operations | experimental economics...,"Spiliotopoulou, Eirini; Conte, Anna",Decision Sciences,2022,https://doi.org/10.1111/deci.12540,We study fairness ideals in distribution syste...,study fairness ideal distribution system inven...


In [59]:
papers_df.loc[25,'abstract_lem']

'consider order allocation game manufacturer source two supplier due supply capacity limitation allocates uneven order take advantage incremental quantity discount laboratory experiment conducted examine empirical decision three member dual sourcing channel observe supplier receives large order mostly agrees supply however supplier receives small order frequently refuse supply consequently manufacturer experience severe supply shortage hurt profit supply chain efficiency develop behavioral model explain empirical decision find fairness concern small order supplier cause supply rejection shortage mitigate behavioral supply risk apply modeling result design subsidy mechanism conduct validation experiment experiment show proposed subsidy mechanism successfully reduces supply rejection channel efficiency significantly enhanced additional cost manufacturer study demonstrates viability behavioral mechanism design approach addressing issue empirical decision making offer insight manufacturer 

## 4. Comparison

In [60]:
top_papers_glove

[(25, 0.9106458322171878),
 (73, 0.8996654499636186),
 (22, 0.8905082589509502),
 (211, 0.876241082404299),
 (247, 0.8664214155713326),
 (223, 0.8615395669748862),
 (242, 0.8603340551374768),
 (115, 0.8392316640413046),
 (104, 0.8387429465784567),
 (3, 0.8372247528261774)]

In [61]:
top_papers_tf_idf

[(211, 0.5657141381232188),
 (224, 0.5615318670172559),
 (247, 0.5428019210807384),
 (246, 0.490500032036113),
 (183, 0.46999268018039),
 (241, 0.46791249380506794),
 (73, 0.46169240302807246),
 (235, 0.45934964982354143),
 (44, 0.4517904067819405),
 (12, 0.44138576774310123)]

## <span style="color:green">5. Vector database</span>

In [62]:
# Load configuration
con_file = open("config.json")
config = json.load(con_file)
con_file.close()

PINECONE_API_KEY = config['pineconeapikey']
PINECONE_ENV = config['pineconeenv']
PINECONE_TABLE_NAME = config['pineconetable']

pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_ENV)

In [63]:
# First, check if our index already exists. If it doesn't, we create it
if PINECONE_TABLE_NAME not in pinecone.list_indexes():
    
    # We create a new index
    pinecone.create_index(
        name=PINECONE_TABLE_NAME,
        metric="cosine",
        dimension=128
    )

In [64]:
# pinecone.delete_index(PINECONE_TABLE_NAME)

In [65]:
index = pinecone.Index(PINECONE_TABLE_NAME)
index.describe_index_stats()

{'dimension': 128,
 'index_fullness': 0.00282,
 'namespaces': {'': {'vector_count': 282}},
 'total_vector_count': 282}

In [66]:
index.describe_index_stats()['dimension']

128

In [67]:
# len(tf_idf.toarray()[0,:].tolist())

In [68]:
# index.upsert([
#     ('1', tf_idf.toarray()[0,:].tolist())
# ])

### Inserting data

In [69]:
batches = int(tf_idf.shape[0]/100)
tf_idf.shape[0]%100

82

In [70]:
# for i in range(batches):
#     batch = []
    
#     for j in range(i*100, (i+1)*100):
#         vector = {}
#         vector['id'] = str(j+1)
#         vector['values'] = tf_idf.toarray()[j,:].tolist()
#         metadata = {}
#         metadata['doi'] = papers_df.loc[j, 'doi']
#         vector['metadata'] = metadata
#         batch.append(vector)

#     index.upsert(batch)

In [71]:
# batch = []
# for i in range(batches*100, batches*100 + tf_idf.shape[0]%100):
#     vector = {}
#     vector['id'] = str(i+1)
#     vector['values'] = tf_idf.toarray()[i,:].tolist()
#     metadata = {}
#     metadata['doi'] = papers_df.loc[i, 'doi']
#     vector['metadata'] = metadata
#     batch.append(vector)
# index.upsert(batch)

### Query an user's input

In [72]:
user_input = "supply chain contract design considering decision maker behavior"
user_input_prepared = nlp_preparation(user_input)
query = tf_idf_vec.transform([user_input_prepared])
query = query.toarray()[:].tolist()[0]

In [73]:
xc = index.query(query, top_k=5, include_metadata=True)
xc

{'matches': [{'id': '212',
              'metadata': {'doi': 'https://doi.org/10.1111/poms.12343'},
              'score': 0.563210547,
              'values': []},
             {'id': '225',
              'metadata': {'doi': 'https://doi.org/10.1111/poms.12057'},
              'score': 0.562829256,
              'values': []},
             {'id': '248',
              'metadata': {'doi': 'https://doi.org/10.1016/j.ijpe.2012.05.004'},
              'score': 0.540547788,
              'values': []},
             {'id': '247',
              'metadata': {'doi': 'https://doi.org/10.1016/j.ijpe.2012.10.023'},
              'score': 0.490448028,
              'values': []},
             {'id': '184',
              'metadata': {'doi': 'https://doi.org/10.1007/s11573-015-0790-4'},
              'score': 0.469891846,
              'values': []}],
 'namespace': ''}

In [74]:
xc['matches'][0]['id']

'212'

In [75]:
xc['matches'][0]['metadata']['doi']

'https://doi.org/10.1111/poms.12343'

In [76]:
xc['matches'][1]

{'id': '225',
 'metadata': {'doi': 'https://doi.org/10.1111/poms.12057'},
 'score': 0.562829256,
 'values': []}

In [77]:
papers_df.head(2)

,title,keywords,authors,journal,publication_year,doi,abstract,abstract_lem
0,Pricing decisions for a social comparison prod...,Behavioral operations management | Pricing dec...,"Zhang, Mengyao; Gou, Qinglong; Yu, Lili; Zhang...",Transportation Research Part E: Logistics and ...,2022,https://doi.org/10.1016/j.tre.2022.102934,As a natural psychological tendency of human b...,natural psychological tendency human being soc...
1,Quantifying heterogeneity in human behavior: A...,behavioral operations management | forklift op...,"Loske, D.; Klumpp, M.; Klumpp, Matthias",Logistics Research,2022,https://doi.org/10.23773/2022_1,Human operators will remain to play an essenti...,human operator remain play essential role pick...


In [78]:
for paper in xc['matches']:
    condition = papers_df['doi'] == paper['metadata']['doi']
    title, doi = papers_df.loc[condition, 'title'].values[0], papers_df.loc[condition, 'doi'].values[0]
    print(title, doi)

Decision Making and Cognition in Multi-Echelon Supply Chains: An Experimental Study https://doi.org/10.1111/poms.12343
Supply chain contract design: Impact of bounded rationality and individual heterogeneity https://doi.org/10.1111/poms.12057
The impact of repeated interactions on supply chain contracts: A laboratory study https://doi.org/10.1016/j.ijpe.2012.05.004
How can we improve the performance of supply chain contracts? An experimental study https://doi.org/10.1016/j.ijpe.2012.10.023
Strategic risk in supply chain contract design https://doi.org/10.1007/s11573-015-0790-4
